In [1029]:
import pandas as pd
from binance.client import Client
from Binance_api import api_key, secret_key
from datetime import datetime
from datetime import timedelta
import time
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
import plot_utils 
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

warnings.filterwarnings('ignore')

client = Client(api_key, secret_key,tld='us')
rsi = 0
buy = 0
sell = 0

In [1030]:
df = pd.read_csv('BTC_5min_10MAR.csv')

In [1031]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band

symbols = ['BTC']

nstd = 1.5
def rsi(symbol, i):
    if df[f'{symbol}_rsi'].iloc[i] > rsi:
        rsi = df[f'{symbol}_rsi'].iloc[i]
    return rsi

def diverg(df,i,symbol):
    p = i - 15
    while p > i - 200:
        if df[f'{symbol}-USD_Close'].iloc[p] < df[f'{symbol}-USD_Open'].iloc[p]:
            if df[f'{symbol}-USD_Close'].iloc[i] < df[f'{symbol}-USD_Close'].iloc[p] and df[f'{symbol}_rsi'].iloc[i] > df[f'{symbol}_rsi'].iloc[p]:
                return True
            else: 
                p = p - 1
        else: 
            p = p - 1


for symbol in symbols:
    df[f'{symbol}_sma'] = sma(df[f'{symbol}-USD_Open'],20)
    df[f'{symbol}_upper_band'], df[f'{symbol}_lower_band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_sma'], 20, nstd)        
    df[f'{symbol}_rsi'] = ta.RSI(df[f'{symbol}-USD_Close'])

        
df.dropna(inplace=True)

In [1032]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.divs = []
        self.trading_fee_multiplier = trading_fee_multiplier
        
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
    
        
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append([self.balance_unit, time, sell_price])
        self.balance_unit = 'USDT'
        
# VIP level 0, paying fees with BNB = 0.075%

        

In [939]:
tickers = ['BTCUSDT','DOGEUSDT']

adabtc_trades = client.get_my_trades(symbol=tickers[0])
adabtc_trades_df = pd.DataFrame(adabtc_trades)

BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action.

In [1033]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=1) #0.99925

In [1034]:
last_buy = 0

for i in range(len(df)):
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
                if diverg(df, i, symbol): 
                 buy = 1
            if  buy == 1 and df[f'{symbol}-USD_Close'].iloc[i] > df[f'{symbol}-USD_Open'].iloc[i]:
                env.buy(symbol, df[f'{symbol}-USD_Close'].iloc[i], df['CloseTime'].iloc[i])
                last_buy = df[f'{symbol}_lower_band'].iloc[i]
                buy = 0
                break
            
    if env.balance_unit != 'USDT':
        if last_buy*0.98 > df[f'{env.balance_unit}-USD_Close'].iloc[i]:
            env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i])
            sell = 0
        elif df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] > : #sell signal
            sell = 1
        if sell == 1 and df[f'{symbol}-USD_Close'].iloc[i] < df[f'{symbol}-USD_Open'].iloc[i]:
            env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[i], df['CloseTime'].iloc[i])
            sell = 0
            
if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])

In [1035]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print('starting balance: 100 USDT')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')

num buys: 623
num sells: 623
starting balance: 100 USDT
ending balance: 42.951000028545906 USDT


In [1020]:
balance = 100
profit_total = 0
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(f'buys = {buys}')
    print(f'sells = {sells}')
    print('P/L = 'f'{profit}')
    balance += balance*(profit/100)
    print(balance)
    print('\n')

buys = ['BTC', '2022-03-10 12:14:59.999', 40889.61]
sells = ['BTC', '2022-03-10 14:30:00', 39743.27]
P/L = -2.80349947089249
97.19650052910751


buys = ['BTC', '2022-03-10 14:54:59.999', 39582.2]
sells = ['BTC', '2022-03-10 18:10:00', 38957.11]
P/L = -1.5792199524028439
95.66155399971451


buys = ['BTC', '2022-03-10 18:19:59.999', 38984.73]
sells = ['BTC', '2022-03-10 23:39:59.999', 39312.87]
P/L = 0.8417141788592541
96.46675086344722


buys = ['BTC', '2022-03-10 23:49:59.999', 38736.01]
sells = ['BTC', '2022-03-11 17:09:59.999', 39111.43]
P/L = 0.9691757101466969
97.40168318118347


buys = ['BTC', '2022-03-11 23:54:59.999', 39292.0]
sells = ['BTC', '2022-03-12 06:45:00', 38364.53]
P/L = -2.360455054464017
95.1025602274002


buys = ['BTC', '2022-03-12 07:04:59.999', 38598.82]
sells = ['BTC', '2022-03-13 03:09:59.999', 39308.59]
P/L = 1.838838596620307
96.85134281123571


buys = ['BTC', '2022-03-13 04:29:59.999', 39115.59]
sells = ['BTC', '2022-03-14 08:45:00', 37903.0]
P/L = -3.1000171